In [1]:
import time
import random
import torch

In [2]:
vocab = [char for char in ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя']

def encrypt(text, key):
    """Returns the encrypted form of 'text'."""
    indexes = [vocab.index(char) for char in text]
    encrypted_indexes = [(idx + key) % len(vocab) for idx in indexes]
    encrypted_chars = [vocab[idx] for idx in encrypted_indexes]
    encrypted = ''.join(encrypted_chars)
    return encrypted

In [3]:
index2char = list(sorted(set(vocab)))
char2index = {c: i for i, c in enumerate(index2char)}
print(index2char, char2index)

[' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё'] {' ': 0, 'а': 1, 'б': 2, 'в': 3, 'г': 4, 'д': 5, 'е': 6, 'ж': 7, 'з': 8, 'и': 9, 'й': 10, 'к': 11, 'л': 12, 'м': 13, 'н': 14, 'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19, 'у': 20, 'ф': 21, 'х': 22, 'ц': 23, 'ч': 24, 'ш': 25, 'щ': 26, 'ъ': 27, 'ы': 28, 'ь': 29, 'э': 30, 'ю': 31, 'я': 32, 'ё': 33}


In [4]:
import io
path = "D:\\onegin.txt"
lines = []

max_len = 0
with io.open(path, encoding='utf-8') as file:
    for line in file:
        line_x = ' '.join(line.lower().translate({ord(c): None for c in 'áâéê!*@#$,.;’-–—1234567890:«»[]?abcdefghijkl…mnopqrstuvwxyz()\'\"'}).split())
        #line_x = line_x.translate({ord(c): None for c in ' '})
        if len(line_x) > max_len:
            max_len = len(line_x)
        lines.append(line_x)
lines = list(filter(None, lines))
print(len(lines), max_len)
print(lines[25])
print(encrypt(lines[25], 1)) # шифрование работает как нужно!

5302 40
но боже мой какая скука
опавпзёанпкалблб атлфлб


In [5]:
num_examples = 5000

def dataset(num_examples):

    dataset_in = []
    dataset_out = []
    for x in range(num_examples):
        rnd = random.randint(0, len(lines)-1)
        rnd2 = random.randint(2, 10)
        text_out = lines[rnd]
        text_in = encrypt(lines[rnd], rnd2)
        
        ex_in = [vocab.index(x) for x in text_in]
        ex_out = [vocab.index(x) for x in text_out]
        
        while len(ex_in) < max_len:
            ex_in.append(0)
            ex_out.append(0)
        
        dataset_in.append(torch.tensor(ex_in))
        dataset_out.append(torch.tensor(ex_out))
        
    return torch.stack(dataset_in), torch.stack(dataset_out)

In [6]:
di, do = dataset(num_examples)
di.shape, do.shape

(torch.Size([5000, 40]), torch.Size([5000, 40]))

In [7]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(34, max_len)
        self.rnn = torch.nn.RNN(max_len, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 34)
        #self.predictions = softmax(scores, dim=2)
        
    def forward(self, sentences, state=None):
        ## Здесь применить
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [8]:
model = Network()

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [10]:
def test_network(line_no, key):
    sentence = encrypt(lines[line_no], key)
    sentence1 = [vocab.index(x) for x in sentence]
    while len(sentence1) < max_len:
        sentence1.append(0)
        
    o = model(torch.tensor(sentence1))
    #print(o.shape)
    w = torch.argmax(o, dim=1)
    #print(w.shape, w[:5])
    
    s = ''
    for i in range(0,len(w)):
        s += vocab[ (w[i]) % len(vocab)]

    return (sentence, '= ' + str(lines[line_no]), '> ' +str(s))


In [11]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    #model.train()
    #print(int(len(di) / 100))
    for i in range(int(len(di) / 100)):
        X_batch = di[i * 100:(i + 1) * 100]
        Y_batch = do[i * 100:(i + 1) * 100]
        #print(X_batch[0])
        #print(X_batch.shape, Y_batch.shape)
        optimizer.zero_grad()      
        o = model.forward(X_batch)

        loss = criterion(o.reshape(-1, 34), Y_batch.flatten())
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
        
    if ep % 10 == 0:
        print('')
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        print('------------------------')
        
        s = ''
        for str_out in test_network(20, 3):
            s+= str_out + '\t'
        print(s)
        s = ''
        for str_out in test_network(20, 4):
            s+= str_out + '\t'
        print(s)
        
        s = ''
        for str_out in test_network(21, 3):
            s+= str_out + '\t'
        print(s)
        s = ''
        for str_out in test_network(21, 4):
            s+= str_out + '\t'
        print(s)



Epoch 0. Time: 0.779, Train loss: 2.110
------------------------
псмвжбжбвфгпюшвъзфхрюшвтугело	= мой дядя самых честных правил	> но    е  оанео   оо  о    оое           	
ртнгзвзвгхдрящгыихцсящгуфдёмп	= мой дядя самых честных правил	> и е      н   таталоо оаоо   н           	
нсёжгврзвевыцхнцвкгрзпсё	= когда не в шутку занемог	> ео         тооео     но                 	
отжздгсигёгьчцочглдсиртж	= когда не в шутку занемог	> еи    о а  тоо   о о                    	

Epoch 10. Time: 0.799, Train loss: 0.712
------------------------
псмвжбжбвфгпюшвъзфхрюшвтугело	= мой дядя самых честных правил	> иой д е  таньх четтоых правил           	
ртнгзвзвгхдрящгыихцсящгуфдёмп	= мой дядя самых честных правил	> нои д дабтвоьх четтоых правил           	
нсёжгврзвевыцхнцвкгрзпсё	= когда не в шутку занемог	> ио в  не вачутлу иаоенод                	
отжздгсигёгьчцочглдсиртж	= когда не в шутку занемог	> ио д  не в чтткт и немое                	

Epoch 20. Time: 0.790, Train loss: 0.393
-----------------